[View in Colaboratory](https://colab.research.google.com/github/LeonidSLA/keras_captcha/blob/master/main.ipynb)

In [0]:
import os
os.chdir('keras_captcha')

In [84]:
ls

captcha_params.py    get_train_set.py   main.ipynb
colab process.ipynb  labeled_captchas/  new txt.txt
generated/           load_data.py       __pycache__/


In [0]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

import captcha_params

from keras.models import Sequential,Model,load_model
from keras.layers import Input, Dense, Dropout, Activation, Flatten,Conv2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.optimizers import adamax
from keras.callbacks import Callback,TensorBoard,ModelCheckpoint
from load_data import load_data
import keras
import get_train_set
#import h5py
#from keras.models import model_from_json

In [85]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: ignored

In [0]:
#Generate additional training set to 'generated' folder
for i in range(8000):     
    text, image = get_train_set.gen_captcha_text_and_image(i)


In [0]:
#original data
from load_data import load_data
(X_train, y_train,y_train_text), (X_test, y_test,y_test_text) = load_data(tol_num = 1000,train_num = 700,folder='labeled_captchas')

In [0]:
# Generated data
(X_train_gen, y_train_gen,y_train_text_gen), (X_test_gen, y_test_gen,y_test_text_gen) = load_data(tol_num = 8000,train_num = 8000,folder='generated')

In [0]:
# Add generated data to train set
X_train_gen=np.append(X_train, X_train_gen,axis=0)
y_train_gen=[np.append(y_train[0],y_train_gen[0],axis=0), 
             np.append(y_train[1],y_train_gen[1],axis=0),
             np.append(y_train[2],y_train_gen[2],axis=0),
             np.append(y_train[3],y_train_gen[3],axis=0),
             np.append(y_train[4],y_train_gen[4],axis=0)]
y_train_text_gen=np.append(y_train_text, y_train_text_gen)

In [0]:
# get input image dimensions
img_rows, img_cols = captcha_params.get_height(), captcha_params.get_width()
input_shape = (img_rows, img_cols, 1)

#Number of classes to predict
nb_classes = 36 #9 digits and 26 letters 

kernel_size =(3,3)
pool_size= (2,2)

model_input=Input(shape=input_shape)

x = MaxPooling2D(pool_size=(2, 2))(model_input)

x = Conv2D(36, kernel_size  )(x)
x = Activation('relu')(x)

x = Conv2D(24, kernel_size  )(x)
x = Activation('relu')(x)

x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)
conv_out = Flatten()(x)

x1 = Dense(nb_classes, activation='softmax')(conv_out)
x2 = Dense(nb_classes, activation='softmax')(conv_out)
x3 = Dense(nb_classes, activation='softmax')(conv_out)
x4 = Dense(nb_classes, activation='softmax')(conv_out)
x5 = Dense(nb_classes, activation='softmax')(conv_out)

lst = [x1, x2, x3, x4, x5]



model = Model(inputs=model_input, outputs=lst)

In [21]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 150, 330, 1)  0                                            
__________________________________________________________________________________________________
max_pooling2d_3 (MaxPooling2D)  (None, 75, 165, 1)   0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 73, 163, 36)  360         max_pooling2d_3[0][0]            
__________________________________________________________________________________________________
activation_3 (Activation)       (None, 73, 163, 36)  0           conv2d_3[0][0]                   
__________________________________________________________________________________________________
conv2d_4 (

In [0]:
#Loss function - summ of 5 losses (One per each symbol)

from keras.losses import categorical_crossentropy
def m_losses(y_true,y_pred):
    loss =categorical_crossentropy(y_true[0], y_pred[0])+categorical_crossentropy(y_true[1], y_pred[1])+categorical_crossentropy(y_true[2], y_pred[2])+categorical_crossentropy(y_true[3], y_pred[3])+categorical_crossentropy(y_true[4], y_pred[4])
    
    return loss

In [0]:
# Set Learning rate and compile model
learning_rate = 0.005
model.compile(loss=m_losses,
              optimizer=adamax(lr=learning_rate),
              metrics=['accuracy'])

# checkpoint to save model weights
filepath="/content/drive/best_model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [92]:
#Training on original data set
model.fit(X_train, y_train, batch_size=100, epochs=100,
            verbose=1, validation_data=(X_test,y_test)#,callbacks=callbacks_list
         )

Train on 700 samples, validate on 300 samples
Epoch 1/100
700/700 [==============================] - 41s 59ms/step - loss: 45.5487 - dense_6_loss: 6.7681 - dense_7_loss: 7.6015 - dense_8_loss: 7.9245 - dense_9_loss: 12.8902 - dense_10_loss: 10.3643 - dense_6_acc: 0.6571 - dense_7_acc: 0.6400 - dense_8_acc: 0.6543 - dense_9_acc: 0.5300 - dense_10_acc: 0.6186 - val_loss: 24.8535 - val_dense_6_loss: 1.8931 - val_dense_7_loss: 4.9258 - val_dense_8_loss: 4.6141 - val_dense_9_loss: 9.8874 - val_dense_10_loss: 3.5331 - val_dense_6_acc: 0.8667 - val_dense_7_acc: 0.7833 - val_dense_8_acc: 0.8100 - val_dense_9_acc: 0.6967 - val_dense_10_acc: 0.8267
Epoch 2/100
700/700 [==============================] - 39s 56ms/step - loss: 18.5285 - dense_6_loss: 2.1087 - dense_7_loss: 3.1546 - dense_8_loss: 3.9203 - dense_9_loss: 5.6185 - dense_10_loss: 3.7265 - dense_6_acc: 0.9443 - dense_7_acc: 0.8757 - dense_8_acc: 0.8414 - dense_9_acc: 0.7543 - dense_10_acc: 0.8886 - val_loss: 18.9078 - val_dense_6_loss: 0

200/700 [=======>......................] - ETA: 24s - loss: 6.5078 - dense_6_loss: 0.0438 - dense_7_loss: 0.3721 - dense_8_loss: 0.0491 - dense_9_loss: 5.9557 - dense_10_loss: 0.0872 - dense_6_acc: 0.9900 - dense_7_acc: 0.9700 - dense_8_acc: 0.9850 - dense_9_acc: 0.9500 - dense_10_acc: 0.9950    700/700 [==============================] - 39s 56ms/step - loss: 4.3102 - dense_6_loss: 2.3153 - dense_7_loss: 0.1238 - dense_8_loss: 0.0477 - dense_9_loss: 1.7646 - dense_10_loss: 0.0589 - dense_6_acc: 0.9843 - dense_7_acc: 0.9743 - dense_8_acc: 0.9671 - dense_9_acc: 0.9557 - dense_10_acc: 0.9857 - val_loss: 10.5436 - val_dense_6_loss: 7.2579e-04 - val_dense_7_loss: 2.9067 - val_dense_8_loss: 1.5588 - val_dense_9_loss: 5.9247 - val_dense_10_loss: 0.1527 - val_dense_6_acc: 0.9900 - val_dense_7_acc: 0.9300 - val_dense_8_acc: 0.9567 - val_dense_9_acc: 0.9267 - val_dense_10_acc: 0.9800
Epoch 20/100
700/700 [==============================] - 39s 56ms/step - loss: 4.1150 - dense_6_loss: 2.3247 - den

100/700 [===>..........................] - ETA: 29s - loss: 0.0112 - dense_6_loss: 3.1190e-04 - dense_7_loss: 6.3181e-06 - dense_8_loss: 5.2691e-05 - dense_9_loss: 8.7866e-04 - dense_10_loss: 0.0100 - dense_6_acc: 0.9700 - dense_7_acc: 0.9700 - dense_8_acc: 0.9600 - dense_9_acc: 0.9500 - dense_10_acc: 0.9700700/700 [==============================] - 39s 56ms/step - loss: 9.6871 - dense_6_loss: 4.6060 - dense_7_loss: 2.4997 - dense_8_loss: 0.1177 - dense_9_loss: 2.4436 - dense_10_loss: 0.0202 - dense_6_acc: 0.9871 - dense_7_acc: 0.9786 - dense_8_acc: 0.9743 - dense_9_acc: 0.9629 - dense_10_acc: 0.9871 - val_loss: 7.6138 - val_dense_6_loss: 2.0921e-04 - val_dense_7_loss: 1.9740 - val_dense_8_loss: 0.3758 - val_dense_9_loss: 4.9968 - val_dense_10_loss: 0.2670 - val_dense_6_acc: 0.9867 - val_dense_7_acc: 0.9500 - val_dense_8_acc: 0.9733 - val_dense_9_acc: 0.9300 - val_dense_10_acc: 0.9767
Epoch 32/100
700/700 [==============================] - 39s 56ms/step - loss: 8.4081 - dense_6_loss: 0

200/700 [=======>......................] - ETA: 24s - loss: 0.0936 - dense_6_loss: 0.0028 - dense_7_loss: 0.0345 - dense_8_loss: 0.0041 - dense_9_loss: 0.0457 - dense_10_loss: 0.0066 - dense_6_acc: 0.9900 - dense_7_acc: 0.9950 - dense_8_acc: 0.9900 - dense_9_acc: 0.9750 - dense_10_acc: 0.9950700/700 [==============================] - 39s 56ms/step - loss: 11344.1780 - dense_6_loss: 2066.5516 - dense_7_loss: 2122.5702 - dense_8_loss: 2605.4751 - dense_9_loss: 2108.0811 - dense_10_loss: 2441.5005 - dense_6_acc: 0.9871 - dense_7_acc: 0.9800 - dense_8_acc: 0.9729 - dense_9_acc: 0.9757 - dense_10_acc: 0.9886 - val_loss: 7.8433 - val_dense_6_loss: 5.2076e-05 - val_dense_7_loss: 1.7962 - val_dense_8_loss: 0.2832 - val_dense_9_loss: 5.7087 - val_dense_10_loss: 0.0552 - val_dense_6_acc: 0.9900 - val_dense_7_acc: 0.9467 - val_dense_8_acc: 0.9667 - val_dense_9_acc: 0.9333 - val_dense_10_acc: 0.9767
Epoch 44/100
700/700 [==============================] - 39s 56ms/step - loss: 11342.9156 - dense_6_

KeyboardInterrupt: ignored

In [103]:
# Training on generated dataset

model.fit(X_train_gen, y_train_gen, batch_size=100, epochs=1,
            verbose=1, validation_data=(X_test,y_test)#,callbacks=callbacks_list
         )

Train on 8000 samples, validate on 300 samples
Epoch 1/1
8000/8000 [==============================] - 397s 50ms/step - loss: 60.6836 - dense_6_loss: 8.3891 - dense_7_loss: 11.8750 - dense_8_loss: 12.8806 - dense_9_loss: 14.2166 - dense_10_loss: 13.3223 - dense_6_acc: 0.5267 - dense_7_acc: 0.3529 - dense_8_acc: 0.3114 - dense_9_acc: 0.2844 - dense_10_acc: 0.2914 - val_loss: 25.5097 - val_dense_6_loss: 1.6103 - val_dense_7_loss: 6.3801 - val_dense_8_loss: 3.4068 - val_dense_9_loss: 11.6785 - val_dense_10_loss: 2.4339 - val_dense_6_acc: 0.9233 - val_dense_7_acc: 0.7767 - val_dense_8_acc: 0.8467 - val_dense_9_acc: 0.7633 - val_dense_10_acc: 0.8467


In [38]:
# Save and load model if necessary 
model.save('best_model.h5')
model.save_weights('best_model_weights.h5')

#model=load_model('best_model.h5', custom_objects={'m_losses': m_losses} )

In [0]:
#Get accuracy on test  sample  separate for each of 5 symbol (first 6 outputs are loss values)
accuracy_scores = model.evaluate(X_test, y_test,verbose=0)[6:11]

In [99]:
#Accuracy score per each captcha symbol 
print(np.round(accuracy_scores,2) )

[0.99 0.96 0.97 0.95 0.98]
